In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/COVID-19/GSE243348'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Longitudinal gene expression profiling of self-collected blood samples in COVID-19+ and healthy participants"
!Series_summary	"Longitudinal cohort: 773 host response genes were profiled in previously vaccinated (n=16) and unvaccinated (n=14) COVID-19+ participants along with 5 healthy uninfected controls across a 2-week observational window"
!Series_summary	"Single timepoint cohort: 773 host response genes were profiled in 6 healthy uninfected participants"
!Series_overall_design	"Longitudinal cohort: 30 COVID-19+ and 5 uninfected participants were asked perform self-collection and stabilization of capillary blood using a novel technology (homeRNA) every other day for two weeks (7 longtiudinal timepoints per participant). Temporal kinetics of 773 immune genes were profiled using the nCounter direct digital counting of native mRNA."
!Series_overall_design	"Single timepoint cohort: 6 healthy uninfected participants were asked perform self-collection

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = 3
gender_row = 2 

def convert_trait(value):
    v = value.split(":")[1].strip()
    if v == 'COVID-19+':
        return 1
    elif v == 'Healthy uninfected':
        return 0
    return None

def convert_age(value):
    v = value.split(":")[1].strip()
    try:
        return float(v)
    except:
        return None

def convert_gender(value):
    v = value.split(":")[1].strip()
    if v == 'female':
        return 0
    elif v == 'male':
        return 1
    return None

save_cohort_info('GSE243348', './preprocessed/COVID-19/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'COVID-19', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/COVID-19/trait_data/GSE243348.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM7783810': [1.0, 44.0, 0.0], 'GSM7783811': [1.0, 44.0, 0.0], 'GSM7783812': [1.0, 44.0, 0.0], 'GSM7783813': [1.0, 44.0, 0.0], 'GSM7783814': [1.0, 44.0, 0.0], 'GSM7783815': [1.0, 44.0, 0.0], 'GSM7783816': [1.0, 44.0, 0.0], 'GSM7783817': [1.0, 29.0, 0.0], 'GSM7783818': [1.0, 29.0, 0.0], 'GSM7783819': [1.0, 29.0, 0.0], 'GSM7783820': [1.0, 29.0, 0.0], 'GSM7783821': [1.0, 29.0, 0.0], 'GSM7783822': [1.0, 29.0, 0.0], 'GSM7783823': [1.0, 51.0, 0.0], 'GSM7783824': [1.0, 51.0, 0.0], 'GSM7783825': [1.0, 51.0, 0.0], 'GSM7783826': [1.0, 51.0, 0.0], 'GSM7783827': [1.0, 51.0, 0.0], 'GSM7783828': [1.0, 51.0, 0.0], 'GSM7783829': [1.0, 51.0, 0.0], 'GSM7783830': [1.0, 32.0, 1.0], 'GSM7783831': [1.0, 32.0, 1.0], 'GSM7783832': [1.0, 32.0, 1.0], 'GSM7783833': [1.0, 32.0, 1.0], 'GSM7783834': [1.0, 32.0, 1.0], 'GSM7783835': [1.0, 32.0, 1.0], 'GSM7783836': [1.0, 32.0, 1.0], 'GSM7783837': [1.0, 32.0, 0.0], 'GSM7783838': [1.0, 32.0, 0.0], 'GSM7783839': [1.0, 32.0, 0.0], 'GSM7783840': [1.0, 32.0, 0.0], 'GSM778

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1', 'ACSL1', 'ACSL3', 'ACSL4',
       'ACVR1', 'ADAR', 'ADGRE5', 'ADGRG3', 'ADORA2A', 'AGT', 'AHR', 'AIF1',
       'AIM2', 'AKT1', 'AKT2', 'AKT3'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1'], 'GB_ACC': ['NM_000789.2', 'NM_001296.5', 'NM_020311.1', 'NM_016557.2', 'NM_001185039.1'], 'PROBE ID - v1.0': ['NM_000789.2:2115', 'NM_001296.5:741', 'NM_020311.1:375', 'NM_016557.2:854', 'NM_001185039.1:1094'], 'PROBE ID - v1.1': ['NM_000789.2:2115', 'NM_001296.5:741', 'NM_020311.1:375', 'NM_016557.2:854', 'NM_001185039.1:1094'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Skipping as requires_gene_mapping is False: Unknown Step

In [7]:

# Moving forward to the next tasks without implementing STEP6 as requires_gene_mapping is False.


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/COVID-19/gene_data/GSE243348.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'COVID-19')

# 4. Save the cohort information.
save_cohort_info('GSE243348', './preprocessed/COVID-19/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/COVID-19/GSE243348.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'COVID-19', the least common label is '0.0' with 37 occurrences. This represents 15.61% of the dataset.
The distribution of the feature 'COVID-19' in this dataset is fine.

Quartiles for 'Age':
  25%: 30.0
  50% (Median): 36.0
  75%: 43.0
Min: 24.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 77 occurrences. This represents 32.49% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/COVID-19/cohort_info.json
